# Importing Necessary Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model

2024-07-02 08:32:54.136633: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 08:32:54.136749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 08:32:54.247589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
trainpath = '/kaggle/input/riceleafdiseasedataset/dataset/train'
testpath = '/kaggle/input/riceleafdiseasedataset/dataset/test'

# Image Processing - Training Data

In [4]:
# Define the new size for resizing images
new_size = 224

# Initialize lists to store images and labels
train_images = []
train_labels = []

# Dictionary mapping class names to numeric labels
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

# Iterate through each subdirectory in the trainpath directory
for i in os.listdir(trainpath):
    # Check if the subdirectory is a disease class
    if i in class_disease:
        print("Entering the folder:", i)

        # Get all image files (.jpg and .JPG) in the current subdirectory
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))

        # Process each image in the subdirectory
        for j in files:
            # Read and preprocess the image
            image_raw = cv2.imread(j)  # Read image in BGR format
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            resize_image = cv2.resize(image, (new_size, new_size))  # Resize image to new_size x new_size

            # Append resized image to train_images list
            train_images.append(list(resize_image))  # Convert ndarray to list and append

            # Append corresponding label to train_labels list
            train_labels.append(class_disease[i])  # Append label based on class_disease dictionary


Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400


# Image Processing - Testing Data

In [5]:
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):  # Entering the test folder
  if i in class_disease:
    print("Entering to the folder name:", i)
    files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) +  gb.glob(pathname=str(testpath + '/' + i + '/*.JPG')) # Pointing to all the .jpg extension image folder
    print("Number of images in the folder is", len(files))
    for j in files:
        image_raw = cv2.imread(j)
        image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
        resize_image = cv2.resize(image, (new_size, new_size))
        test_images.append(list(resize_image))
        test_labels.append(class_disease[i])


Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200


In [6]:
def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)


In [7]:
# Function to convert categorical labels into one hot encoded labels
def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

((5132,), (800,))

In [22]:
# Function to convert one-hot encoded labels to categorical labels
def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

In [8]:
# Ensure you're using GPU
gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

if len(gpus) < 2:
    print("Not enough GPUs available, ensure your environment is configured correctly")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

    # Strategy for multi-GPU
    strategy = tf.distribute.MirroredStrategy()

Num GPUs Available: 2


# Deep Feature Extraction - VGG16

In [10]:
# Function to define the model
def model_vgg16():
    VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in VGG_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(VGG_model.output)
    output = Model(inputs=VGG_model.input, outputs=feature)
    return output


# Random Forest Classifier - VGG16 Deep Features

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)

        rf = RandomForestClassifier()
        rf = rf.fit(train_feature_16, y_train)
        test_pred = rf.predict(test_feature_16)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 22s 95ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 1 - Accuracy: 0.95625000, Recall: 0.95625000, Precision: 1.00000000, F1 Score: 0.97676196, AUC: 0.97812500
161/161 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step
Run 2 - Accuracy: 0.96250000, Recall: 0.96250000, Precision: 0.99748744, F1 Score: 0.97908993, AUC: 0.98083333
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step
Run 3 - Accuracy: 0.95000000, Recall: 0.95000000, Precision: 1.00000000, F1 Score: 0.97358457, AUC: 0.97500000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step
Run 4 - Accuracy: 0.96125000, Recall: 0.96125000, Precision: 1.00000000, F1 Score: 0.97947487, AUC: 0.98062500
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step
Run 5 - Accuracy: 0.96375000, Recall: 0.96375000, Precision: 1.00000000, F1 Score: 0.98089

# KNN Classifier - VGG16 Deep Features

In [10]:
from sklearn.neighbors import KNeighborsClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)

        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(train_feature_16, y_train)
        test_pred = knn.predict(test_feature_16)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
Run 1 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98029509, F1 Score: 0.97993278, AUC: 0.98666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 85ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step
Run 2 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98029509, F1 Score: 0.97993278, AUC: 0.98666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step
Run 3 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98029509, F1 Score: 0.97993278, AUC: 0.98666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step
Run 4 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98029509, F1 Score: 0.97993278, AUC: 0.98666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step
Run 5 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98029509, F1 Score: 0.97993278, AUC: 0.98666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# SVM Classifier - VGG16 Deep Features

In [20]:
from sklearn.svm import SVC

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        svm = SVC(probability=True)
        svm.fit(train_feature_16, y_train_cat)
        test_pred = svm.predict(test_feature_16)
        test_pred_proba = svm.predict_proba(test_feature_16)

        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 87ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step
Run 1 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
Run 2 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
Run 3 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step
Run 4 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 87ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
Run 5 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 

# Decision Tree Classifier - VGG16

In [24]:
from sklearn.tree import DecisionTreeClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)

        # Define and train Decision Tree Classifier
        dt = DecisionTreeClassifier()
        dt.fit(train_feature_16, y_train)
        test_pred = dt.predict(test_feature_16)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step
Run 1 - Accuracy: 0.91250000, Recall: 0.91250000, Precision: 0.91488474, F1 Score: 0.91195653, AUC: 0.94166667
161/161 ━━━━━━━━━━━━━━━━━━━━ 15s 89ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step
Run 2 - Accuracy: 0.91000000, Recall: 0.91000000, Precision: 0.91364912, F1 Score: 0.90902413, AUC: 0.94000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
Run 3 - Accuracy: 0.91875000, Recall: 0.91875000, Precision: 0.91952491, F1 Score: 0.91814294, AUC: 0.94583333
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step
Run 4 - Accuracy: 0.91500000, Recall: 0.91500000, Precision: 0.91712056, F1 Score: 0.91436076, AUC: 0.94333333
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
Run 5 - Accuracy: 0.91250000, Recall: 0.91250000, Precision: 0.91376676, F1 Score: 0.91185261, AUC: 0.94166667
161/161 ━━━━━━━━━━━━━━━━━━━━ 1

# XGBoost Classifier - Deep Features VGG16

In [18]:
from xgboost import XGBClassifier

# Function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():
        model_FE_16 = model_vgg16()
        model_FE_16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

        train_feature_16 = model_FE_16.predict(X_train)
        test_feature_16 = model_FE_16.predict(X_test)

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        # Define and train XGBoost Classifier
        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(train_feature_16, y_train_cat)
        test_pred = xgb.predict(test_feature_16)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, xgb.predict_proba(test_feature_16), multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")


161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step
Run 1 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 86ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step
Run 2 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
Run 3 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step
Run 4 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step
Run 5 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 0.99630542, F1 Score: 0.99624979, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 1